![](CRISP_DM.png)

In [ ]:
# importing requires 
import pandas as pd
import numpy as np
# for plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
print('Your current Plotly version is:' +plotly.__version__)#checking current plotly version
import plotly.graph_objects as go
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# setup parameters for graph plotting
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

In [ ]:
# importing requiring packages for dynamic dashboard
import dash
print('Your current version of Dash package is: ' +dash.__version__)
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

## 5.1 Explorative Data Analysis
* In statistics, exploratory data analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modeling or hypothesis testing task.<font color=red> Source:Wiki</font> 

In [ ]:
# importing CSV file as dataframe with date in ascending order
df_EDA=pd.read_csv('C:/Users/dhame/ds_covid-19/data/processed/COVID_small_flat_table.csv',sep=';')
df_EDA.sort_values('date',ascending=False).head()

### 5.1.1 Plotting using Matplotlib package

In [ ]:
ax=df_EDA.iloc[15:,:].set_index('date').plot()
plt.ylim(10, 40000)
ax.set_yscale('log')
plt.show();

In [ ]:
# save fig in local drive
plt.savefig('C:/Users/dhame/ds_covid-19/data/processed/plot_of_5_countries.jpg')

### 5.1.2 Plotting using Plotly package

In [ ]:
country_list=['Italy','US','Spain','Germany','Korea, South'] 

In [ ]:
fig = go.Figure()
for each in country_list:
    fig.add_trace(go.Scatter(x=df_EDA.date,y=df_EDA[each],mode='markers+lines', opacity=1.0,line_width=2.5,marker_size=4,name=each))

fig.update_layout(width=700,height=500,xaxis_title="Timeline in days",
    yaxis_title="Confirmed infected people (Source:Johns-hopkins csse, log-scale)")
fig.update_yaxes(type="log",range=[1.1,5.5])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

### 5.1.3 Development of dynamic and interactiveDashboard

In [ ]:
# Create layout for dashboard for ex. Dropdown menu, multiple selection enabled etc.
app = dash.Dash()
app.layout = html.Div([
    
    html.Label('Multiple-selection of Country'),
    dcc.Dropdown(
        id='country_drop_down',
        options=[{'label': 'Italy', 'value': 'Italy'},{'label': 'US', 'value': 'US'},{'label': 'Spain', 'value': 'Spain'},
            {'label': 'Germany', 'value': 'Germany'},{'label': 'Korea, South', 'value': 'Korea, South'}],
        value=['Germany','US'], # Country will be presented in default layout
        multi=True),   
        
    dcc.Graph(figure=fig, id='main_window_slope')])

In [ ]:
# when user selects other parameter than default in dashboard \
# then it makes changes according to that selection
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_EDA.date,y=df_EDA[each],mode='markers+lines',colorscale='Viridis',
                           opacity=1.0,line_width=2.5,marker_size=4, name=each))
        
    return {
            'data': traces,
            'layout': dict (
                width=1024,
                height=800,
                xaxis_title="Timeline in Days",
                yaxis_title="Confirmed infected people (Source:Jhons-hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#e377c2"),},
                yaxis={'type':"log",
                       'range':'[1.1,5.5]'})}

In [ ]:
#Run the app in your browser
app.run_server(debug=True, use_reloader=False)